# Expanding Variable Cluster Timeline

Author: Sofia Ingersoll 5 May 2024

## Increaseing Timeline
Loading files in 4, 500 cluster chunks 
`campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC0113.clm2.h0.1995-02-01-00000.nc`


**Help**
- It's taking too long to complete the task, so I'm not sure these are viable approaches

In [ ]:
import xarray as xr
import pickle
from utils import * 

In [2]:
# Request an additional 10 cores of power for processing from the server
client = get_cluster("UCSB0021", cores = 30)
# apply peer2peer network communication across multiple devices
client.cluster

/glade/u/home/singersoll/.conda/envs/meds-py/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40417 instead
  warnings.warn(


Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/singersoll/proxy/40417/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.67:33915,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/singersoll/proxy/40417/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [3]:
cluster_lists = [
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.1995-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2000-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2005-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2010-02-01-00000.nc'))
    ]

### TAKES TOO LONG
### Trying to improve the speed
Difference here: we load in the data w/ concat on dims 'ens' & 'time',

then preprocess the dimensions & concat using compute 

In [14]:
def read_all_simulations(var):
    '''Prepare cluster list and read to create ensemble(group of data)
    Use preprocess to select only certain dimension and a variable'''
    # Define the list of lists
    cluster_lists = [
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.1995-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2000-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2005-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2010-02-01-00000.nc'))
    ]
    
    all_data = []  # List to store data from all simulations
    
    for cluster_list in cluster_lists:
        # Exclude the first element of the cluster_list
        cluster_list = cluster_list[1:]
        
        # Read each simulation
        ds = xr.open_mfdataset(cluster_list,
                               combine='nested',
                               concat_dim=['ens', 'time'],
                               parallel=True)
        
        # Preprocess the dataset to select only certain dimensions and the variable of interest
        ds = ds[['lat', 'lon', 'time', var]].compute()
        
        # Append the preprocessed and computed dataset to the list
        all_data.append(ds)
    
    # Combine all the datasets into a single dataset along the 'ens' and 'time' dimensions
    combined_ds = xr.concat(all_data, dim=["ens", "time"])
    
    return combined_ds

In [ ]:
# started at 10:19
# giving it till 10:29
# interupted bc took to long
read_all_simulations("LNC")

### TAKES TOO LONG

In [12]:
def read_all_simulations(var):
    '''Prepare cluster list and read to create ensemble(group of data)
    Use preprocess to select only certain dimension and a variable'''
    # Define the list of lists
    cluster_lists = [
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.1995-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2000-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2005-02-01-00000.nc')),
        sorted(glob.glob('/glade/campaign/cgd/tss/projects/PPE/PPEn11_LHC/transient/hist/PPEn11_transient_LHC[0][0-5][0-9][0-9].clm2.h0.2010-02-01-00000.nc'))
    ]
    
    all_data = []  # List to store data from all simulations
    
    for cluster_list in cluster_lists:
        # Exclude the first element of the cluster_list
        cluster_list = cluster_list[1:]
        
        # Read each simulation and append to the list
        ds = xr.open_mfdataset(cluster_list,
                               combine='nested',
                               concat_dim=['ens', 'time'],
                               preprocess=lambda ds: ds[['lat', 'lon', 'time', var]],
                               parallel=True)
        all_data.append(ds)
    
    # Return a list of datasets
    return all_data

In [ ]:
# This takes too long, it's been running for 5 mins
# It's 9:54 now, I'm giving it until 10:05 and if it hasn't loaded
# I'm interupting kernel and trying something diff
# bc we can't use something that takes that long
# got side tracked, it's still not done at 10:17
read_all_simulations("LNC")

Task exception was never retrieved
future: <Task finished name='Task-2815698' coro=<Client._gather.<locals>.wait() done, defined at /glade/u/home/singersoll/.conda/envs/meds-py/lib/python3.11/site-packages/distributed/client.py:2197> exception=AllExit()>
Traceback (most recent call last):
  File "/glade/u/home/singersoll/.conda/envs/meds-py/lib/python3.11/site-packages/distributed/client.py", line 2206, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-2815733' coro=<Client._gather.<locals>.wait() done, defined at /glade/u/home/singersoll/.conda/envs/meds-py/lib/python3.11/site-packages/distributed/client.py:2197> exception=AllExit()>
Traceback (most recent call last):
  File "/glade/u/home/singersoll/.conda/envs/meds-py/lib/python3.11/site-packages/distributed/client.py", line 2206, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-2815592

### Improving Func Integration 
It would be beneficial to move our if else statement outside of the `read_n_wrangle()` and directly into the `read_all_simulations()` because it doesn't need to depend on the parameter / variable combination bc the param data is the same for all sims. 

Next step, adding this into the function above once we're able to improve the computational speed of `read_all_simulations()`

In [ ]:
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #----        If-else Load Data       ----
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    filepath = os.path.join("saves", f"{var}.nc")
    if os.path.exists(filepath):
         #read in the file as a dataset
        ds=xr.open_dataset('saves/'+var+'.nc')
    
        #then convert back to data array
        var_avg = ds[var]
    else:
        print(f"Reading and wrangling your data, this may take a few minutes")